In [1]:
import numpy as np

In [ ]:
# Loading Source Data and Calibration File
fcs = tables.open_file( '/Users/eframe/Downloads/cs.h5', 'r' )
fco = tables.open_file( '/Users/eframe/Downloads/co.h5', 'r' )
detector = f.root.EventData.read()[ 'detector' ]
energy = f.root.EventData.read()[ 'ADC_value' ]

In [ ]:
# Finding Peaks of Gamma-ray Lines for Calibration
Co, Cs = [ ], [ ]
for s in np.arange( 0, 152 ):
    mask = ( detector == s )
    hist, be = np.histogram( energy[mask], bins = 1500, range = ( 10, 1500 ) )
    bins = calib.bin_centers( be )
    maskCo = ( bins > 100 ) & ( bins < 140 )
    maskCs = ( bins > 640 ) & ( bins < 690 )
    
    poptCo, pcovCo = calib.curvefit( bins[ maskCo ], hist[ maskCo ] )
    poptCs, pcovCs = calib.curvefit( bins[ maskCs ], hist[ maskCs ] )
    
    Comax = np.argmax( hist[ maskCo ] )
    Csmax = np.argmax( hist[ maskCs ] )
    
    Co.append( [ s, bins[ maskCo ][Comax] ] )
    Cs.append( [ s, bins[ maskCs ][Csmax] ] )  

In [ ]:
# Linear Fit of Peak Data incorporating Old Calib File
peaks = np.array( [  np.array( Co )[ :, 1 ], np.array( Cs )[ :, 1 ] ] ).T
E = [ 122, 662 ]
newcalib = []
for i in np.arange( 0, 152 ):
    if np.sum( peaks[ i ] ) > 0:
        p = np.polyfit( peaks[ i ], E, 1 )
        newcalib.append( [ b, m ] )
    else:
        newcalib.append( [ 0.000000000000000000e+00, 0.000000000000000000e+00 ] )
np.savetxt( '/Users/eframe/Downloads/CCI2_Calibration_9-23-21.txt', np.array( newcalib ), delimiter=' ' )

In [ ]:
# Build Calibration File to Read out ADC Values
newcalib = []
for i in np.arange( 0, 152 ):
    newcalib.append( [ 0.000000000000000000e+00, 1.000000000000000000e+00 ] )
np.savetxt( '/Users/eframe/Downloads/CCI2_ADC_Readout.txt', np.array( newcalib ), delimiter=' ' )